# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

### Week 2 - Applied Data Science Capstone (COURSERA)

##### Aim: to explore and cluster the neighborhoods in Toronto

# Introduction

Data set from Wikipedia page:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Note: There are different website scraping libraries and packages in Python. In this notebook, we simply use pandas to read the table into a pandas dataframe.

#### Download all the dependencies we may need and import packages

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
# convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize 
# tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes
import folium 
# map rendering library

print('Libraries imported.')

Libraries imported.


# Part 1 - Create a Dataframe and Transforme the data from the Wikipedia page into pandas dataframe

#### Step 1 - Using read_html from pandas to read in tables from a html; get a dataframe (df) with three columns: PostalCode, Borough, and Neighborhood

In [2]:
raw_data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
# determine the index of raw_data
# raw_data = raw_data[0]

# Looks like that our table of interest is the first element in the list "raw_data" where we scraped the data to. 
# So, let's create a data frame from that element
df = pd.DataFrame(raw_data[0]) 
df.head() # check the first 5 rows


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Step 2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
# Check how many rows have no assigned borough 
rows_to_ignore = len(df["Borough"][df["Borough"] == "Not assigned"])
print('number of rows without an assigned borough '+str(rows_to_ignore))

number of rows without an assigned borough 77


In [4]:
# There are indeed rows without an assigned borough. let's ignore them
df = df[df.Borough != 'Not assigned']
df.head() # check the first 5 rows

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Step 3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [5]:
# Check if more than one neighborhood exist in one postal code area
print(len(df['Postal Code'].unique()) == len(df['Postal Code']))
print('(if this returns True, it does not exist more than one neighbourhood in one postal code area)')

# If this statement returns True, then all the Postal Codes are unique values and therefore are not listed to different
# neighbourhoods.

# If the statement returns False, uncomment the following line, as it combines the neighbourhoods with the same Postal Code,
# separated with a comma
#df = df.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

True
(if this returns True, it does not exist more than one neighbourhood in one postal code area)


#### Step 4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [6]:
# Check if there is any cell with a Not assigned neighbourhood
print(len(df["Neighbourhood"][df["Neighbourhood"] == "Not assigned"]))
print('(if this returns null, then there is not any cell with a Not assigned neighbourhood)')

# If this returns null, then there is not any cell with a Not assigned neighbourhood
# Else, uncomment and run the following couple of lines:
#    if row['Neighbourhood'] == 'Not assigned':
#        row['Neighbourhood'] = row['Borough']

0
(if this returns null, then there is not any cell with a Not assigned neighbourhood)


#### Step 5. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

In [7]:
#In order to combine the value of "Neighbourhood" that have the same Postcode, 
#I will use .groupby() and join value, separating them by a comma, and overwrite 
#the "toronto" dataframe with the new dataframe where "Neighbourhood"s are listed in one row. 
# Then I'll reset the index and check the first 5 rows
df = pd.DataFrame(df.groupby(["Postal Code", "Borough"])["Neighbourhood"].apply(lambda hoods: ", ".join(hoods)))
df.reset_index(inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Step 6. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [8]:
df.shape

(103, 3)

# Part 2 - in order to utilize the Foursquare location data, get the latitude and the longitude coordinates of each neighborhood

get the latitude and the longitude coordinates of each neighborhood using Geocoder package

In [9]:
# !conda install -c conda-forge geopy
# from geopy.geocoders import Nominatim

# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]


I was not able to use the Geocoder package to get lat and long coordinates for each neighbourhood. So, I had to use the link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [10]:
#Using the csv file to create the following dataframe
import urllib.request
urllib.request.urlretrieve("http://cocl.us/Geospatial_data", "lat_lon_Toronto.csv")

('lat_lon_Toronto.csv', <http.client.HTTPMessage at 0xa89a630>)

In [11]:
lat_lon = pd.read_csv("lat_lon_Toronto.csv")
# Check the first 5 rows
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# merge the lat and long with the previous dataframe
df = pd.merge(lat_lon, df, on='Postal Code')
df.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [13]:
# reorder column names and show the dataframe
df = df[['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
df.shape

(103, 5)

# Part 3 - Clustering the Neighbourhoods¶

In part 3.1 we'll only going to work with boroughs that contain the word Toronto
In part 3.2 we'll explore Neighbourhoods in Toronto

### Part 3.1 - exploring boroughs that contain the word Toronto

In [15]:
# filter the dataframe to have boroughs that contain the word Toronto
filter_toronto = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
filter_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Visualize the boroughs that contain the word Toronto

In [16]:
# create map of Toronto using latitude and longitude values
latitude = filter_toronto.loc[0, 'Latitude']
longitude = filter_toronto.loc[0, 'Longitude']
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(filter_toronto['Latitude'], filter_toronto['Longitude'], filter_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
print('Latitude and longitude values of Toronto are {}, {}.'.format(latitude, 
                                                               longitude))    
map_toronto

Latitude and longitude values of Toronto are 43.67635739999999, -79.2930312.


In [33]:
# Setting Foursquare Credentials and Version

CLIENT_ID = 'XXX' # your Foursquare ID
CLIENT_SECRET = 'XXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XXX
CLIENT_SECRET:XXX


I choose to futher explore first neighbourhood of our dataframe

In [18]:
# Check the neighbourhood's name and respective lat and long

neighborhood_name = filter_toronto.loc[0, 'Neighbourhood'] # neighbourhood name
neighborhood_latitude = filter_toronto.loc[0, 'Latitude'] # neighbourhood latitude
neighborhood_longitude = filter_toronto.loc[0, 'Longitude'] # neighbourhood longitude


print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


Next, we are exploring the top 10 venues that are in the first neighbourhood of our dataframe (The Beaches) within a radius of 750 meters

In [19]:
# set the URI (Uniform Resource Identifier)

LIMIT = 10 # Set the number of venues that we wish to return from Foursquare API
radius = 750 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get request and check the result
out_request = requests.get(url).json()
out_request

{'meta': {'code': 200, 'requestId': '5f241696d79d8e3962b2e44b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 43.68310740675, 'lng': -79.28371577173579},
   'sw': {'lat': 43.66960739324999, 'lng': -79.30234662826422}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'di

From the Foursquare lab , we know that all the information is in the items key. Let's extract the category of the venue. Then, we are ready to clean the json and structure it into a pandas dataframe

In [32]:
# Extract the category of the venue
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

# Clean the JSON and structure it into a *pandas* dataframe    
venues = out_request['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()    

C:\Users\ana roque\anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Beech Tree,Gastropub,43.680493,-79.288846
2,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331
3,Beaches Bake Shop,Bakery,43.680363,-79.289692
4,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993


### Part 3.2 - exploring neighbourhoods in Toronto

Create a function to repeat the same process to all the neighbourhoods in Toronto

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)   
    
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


# run the above function on each neighborhood and create a new dataframe called *toronto_venues*
toronto_venues = getNearbyVenues(names=filter_toronto['Neighbourhood'],
                                latitudes=filter_toronto['Latitude'],
                                longitudes=filter_toronto['Longitude']
                                )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [21]:
# check the size of the resulting dataframe and check the first 5 rows
print(toronto_venues.shape)
toronto_venues.head()

(348, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [22]:
# check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 122 uniques categories.


 #### Analyze Each Neighbourhood

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Skating Rink,Spa,Speakeasy,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
# examine the new dataframe size
toronto_onehot.shape

(348, 122)

In [25]:
# let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Skating Rink,Spa,Speakeasy,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.100000,0.0,0.0,0.000,0.0,0.000000,0.0,0.00,0.0,0.0,0.1,0.0,0.0,0.000,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.00,0.1,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000000,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.0,0.1,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.1,0.000,0.0,0.000000,0.0,0.00,0.0,0.0,0.1,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.1,0.0,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.100,0.0,0.000000,0.0,0.00,0.0,0.0,0.1,0.0,0.0,0.000,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.1,0.1,0.2,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.1,0.00,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000,0.1,0.000000,0.0,0.00,0.0,0.1,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
4,Central Bay Street,0.0,0.0

In [26]:
# confirm the new size

toronto_grouped.shape

(39, 122)

In [27]:
# Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0  Farmers Market   0.1
1        Beer Bar   0.1
2    Cocktail Bar   0.1
3          Museum   0.1
4    Liquor Store   0.1


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0         Coffee Shop   0.2
1              Bakery   0.1
2                Café   0.1
3          Restaurant   0.1
4  Italian Restaurant   0.1


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
           venue  freq
0     Comic Shop   0.1
1  Auto Workshop   0.1
2    Pizza Place   0.1
3     Restaurant   0.1
4  Burrito Place   0.1


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0      Airport Lounge   0.2
1                 Bar   0.1
2  Airport Food Court   0.1
3     Airport Service   0.1
4    Airport Terminal   0.1


----Central Bay Street----
                        venue  freq
0                 Coffee Shop 

Let's put that into a *pandas* dataframe

In [28]:
# First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Farmers Market,Park,Concert Hall,Liquor Store,Restaurant,Beer Bar,Museum,Cocktail Bar,Vegetarian / Vegan Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Gym,Bar,Bakery,Italian Restaurant,Restaurant,Pet Store,Furniture / Home Store,Cuban Restaurant
2,"Business reply mail Processing Centre, South C...",Garden Center,Pizza Place,Brewery,Skate Park,Farmers Market,Auto Workshop,Fast Food Restaurant,Burrito Place,Comic Shop,Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Plane,Airport,Airport Food Court,Airport Service,Airport Terminal,Rental Car Location,Harbor / Marina,Bar,Vietnamese Restaurant
4,Central Bay Street,Coffee Shop,Sushi Restaurant,Modern European Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Gastropub,Fish Market,Department Store,Comic Shop
5,Christie,Café,Grocery Store,Coffee Shop,Diner,Restaurant,Candy Store,Italian Restaurant,Department Store,Distribution Center,Dessert Shop
6,Church and Wellesley,Salon / Barbershop,Park,Beer Bar,Bookstore,Ramen Restaurant,Breakfast Spot,Bubble Tea Shop,Dance Studio,Theme Restaurant,Restaurant
7,"Commerce Court, Victoria Hotel",Café,Restaurant,Museum,Japanese Restaurant,Bakery,Gym / Fitness Center,Gym,Coffee Shop,American Restaurant,Dance Studio
8,Davisville,Dessert Shop,Indian Restaurant,Coffee Shop,Seafood Restaurant,Park,Italian Restaurant,Café,Pizza Place,Sushi Restaurant,Dance Studio
9,Davisville North,Sandwich Place,Breakfast Spot,Park,Hotel,Food & Drink Shop,Pizza Place,Department Store,Gym / Fitness Center,Dance Studio,Dessert Shop


#### Cluster Neighborhoods

In [30]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([0, 1, 0, 0, 1, 1, 0, 1, 0, 0])

In [31]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

toronto_merged = filter_toronto

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Vietnamese Restaurant,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega,Cuban Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Ice Cream Shop,Cosmetics Shop,Juice Bar,Brewery,Italian Restaurant,Yoga Studio,Flea Market,Fish Market,Concert Hall
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Ice Cream Shop,Sushi Restaurant,Brewery,Pub,Burrito Place,Gym,Italian Restaurant,Fast Food Restaurant,Fish & Chips Shop,Park
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Cheese Shop,Fish Market,Pet Store,Bookstore,Ice Cream Shop,Sandwich Place,Seafood Restaurant,Coffee Shop,Gay Bar,Diner
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Bus Line,Swim School,Dog Run,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant


In [32]:
# Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters